In [1]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [3]:
!pip install transformers
!pip install jsonlines

In [4]:

# 2. Import necessary libraries
import jsonlines
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import os



In [5]:
save_directory = '/content/drive/MyDrive/modelMyCTIBD'
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name =save_directory, # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(save_directory)


In [7]:
from unsloth import FastLanguageModel

# Load the model with 4-bit quantization using `unsloth`
model, _ = FastLanguageModel.from_pretrained( # Assuming the model is the first element of the tuple
    model_name=save_directory,
    dtype=torch.float16,
    load_in_4bit=True
)
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.9.post2: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Unsloth 2024.9.post2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

datasetiiiy

In [8]:
import pandas as pd
import json

# Load the TSV file
df = pd.read_csv('/content/output_questions_20(2).tsv', sep='\t')
df.columns = df.columns.str.strip()

# Create a list to hold the JSON objects
data = []

# Process each row in the DataFrame
for _, row in df.iterrows():
    entry = {
        'question': row['Question'],
        'options': {
            'A': row['Option A'],
            'B': row['Option B'],
            'C': row['Option C'],
            'D': row['Option D']
        },
        'correct_answer': row['Correct Answer']
    }
    data.append(entry)

# Save to JSONL file
with open('/content/dataJson.jsonl', 'w') as f:
    for item in data:
        f.write(json.dumps(item) + '\n')


In [9]:
import json

input_file = "/content/dataJson.jsonl"
output_file = "/content/cleaned_dataJson.jsonl"

# Function to sanitize options (convert all option values to strings)
def sanitize_options(options):
    for key, value in options.items():
        options[key] = str(value)  # Ensure all options are strings
    return options

# Open the original dataset and the output file
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        try:
            # Parse each line as JSON
            example = json.loads(line)

            # Sanitize the 'options' field
            if 'options' in example:
                example['options'] = sanitize_options(example['options'])

            # Write the sanitized example back to the new file
            outfile.write(json.dumps(example) + "\n")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue  # Skip any lines with JSON errors

print("Dataset has been cleaned and saved to", output_file)


Dataset has been cleaned and saved to /content/cleaned_dataJson.jsonl


In [ ]:


# Load the dataset from a JSONL file
dataset_path = '/content/cleaned_dataJson.jsonl'  # Change this path
mcq_dataset = []

with jsonlines.open(dataset_path) as reader:
    for obj in reader:
        mcq_dataset.append(obj)

# Alpaca-style prompt template
alpaca_prompt = "Question: {}\nOptions:\n{}\nAnswer:"

# Function to prepare the prompt
def prepare_prompt(question, options):
    # Ensure options is a dictionary and format correctly
    if isinstance(options, dict):
        options_text = "\n".join([f"{key}: {value}" for key, value in options.items()])
    else:
        options_text = options  # Handle the case where options are already formatted as a string
    return alpaca_prompt.format(question, options_text)

# Evaluation function using generation
def evaluate_mcq_with_generation(question, options, correct_answer, tokenizer, model):
    # Prepare the prompt
    prompt = prepare_prompt(question, options)

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the output and get the predicted answer
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the predicted answer (adapting based on the model's output format)
    if "Answer:" in generated_text:
        predicted_answer = generated_text.split("Answer:")[-1].strip()  # Extracting the answer after "Answer:"
    else:
        predicted_answer = generated_text.strip()  # In case the model doesn't output "Answer:"

    # Compare predicted answer with the correct answer
    return predicted_answer.lower() == correct_answer.lower()  # Case-insensitive comparison

# Evaluate the entire dataset
correct_predictions = 0
total_questions = len(mcq_dataset)

for mcq in mcq_dataset:
    question = mcq.get("question", "")
    options = mcq.get("options", "")
    correct_answer = mcq.get("correct_answer", "")

    # Check if correct_answer is valid before evaluating
    if isinstance(correct_answer, str):
        if evaluate_mcq_with_generation(question, options, correct_answer, tokenizer, model):
            correct_predictions += 1

# Calculate and print accuracy
accuracy = correct_predictions / total_questions
print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")


Evaluation Accuracy: 56.51%


Exemple on other dataset
ayoubData

In [ ]:
import pandas as pd
import json

# Load the TSV file
df = pd.read_csv('/content/DatasetGen.csv')
df.columns = df.columns.str.strip()

# Create a list to hold the JSON objects
data = []

# Process each row in the DataFrame
for _, row in df.iterrows():
    entry = {
        'question': row['Question'],
        'options': {
            'A': row['Option A'],
            'B': row['Option B'],
            'C': row['Option C'],
            'D': row['Option D']
        },
        'correct_answer': row['GT']
    }
    data.append(entry)

# Save to JSONL file
with open('/content/dataJson.jsonl', 'w') as f:
    for item in data:
        f.write(json.dumps(item) + '\n')

In [ ]:
import json

input_file = "/content/dataJson.jsonl"
output_file = "/content/cleaned_dataJson.jsonl"

# Function to sanitize options (convert all option values to strings)
def sanitize_options(options):
    for key, value in options.items():
        options[key] = str(value)  # Ensure all options are strings
    return options

# Open the original dataset and the output file
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        try:
            # Parse each line as JSON
            example = json.loads(line)

            # Sanitize the 'options' field
            if 'options' in example:
                example['options'] = sanitize_options(example['options'])

            # Write the sanitized example back to the new file
            outfile.write(json.dumps(example) + "\n")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue  # Skip any lines with JSON errors

print("Dataset has been cleaned and saved to", output_file)


Dataset has been cleaned and saved to /content/cleaned_dataJson.jsonl


In [ ]:


# Load the dataset from a JSONL file
dataset_path = '/content/cleaned_dataJson.jsonl'  # Change this path
mcq_dataset = []

with jsonlines.open(dataset_path) as reader:
    for obj in reader:
        mcq_dataset.append(obj)

# Alpaca-style prompt template
alpaca_prompt = "Question: {}\nOptions:\n{}\nAnswer:"

# Function to prepare the prompt
def prepare_prompt(question, options):
    # Ensure options is a dictionary and format correctly
    if isinstance(options, dict):
        options_text = "\n".join([f"{key}: {value}" for key, value in options.items()])
    else:
        options_text = options  # Handle the case where options are already formatted as a string
    return alpaca_prompt.format(question, options_text)

# Evaluation function using generation
def evaluate_mcq_with_generation(question, options, correct_answer, tokenizer, model):
    # Prepare the prompt
    prompt = prepare_prompt(question, options)

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the output and get the predicted answer
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the predicted answer (adapting based on the model's output format)
    if "Answer:" in generated_text:
        predicted_answer = generated_text.split("Answer:")[-1].strip()  # Extracting the answer after "Answer:"
    else:
        predicted_answer = generated_text.strip()  # In case the model doesn't output "Answer:"

    # Compare predicted answer with the correct answer
    return predicted_answer.lower() == correct_answer.lower()  # Case-insensitive comparison

# Evaluate the entire dataset
correct_predictions = 0
total_questions = len(mcq_dataset)

for mcq in mcq_dataset:
    question = mcq.get("question", "")
    options = mcq.get("options", "")
    correct_answer = mcq.get("correct_answer", "")

    # Check if correct_answer is valid before evaluating
    if isinstance(correct_answer, str):
        if evaluate_mcq_with_generation(question, options, correct_answer, tokenizer, model):
            correct_predictions += 1

# Calculate and print accuracy
accuracy = correct_predictions / total_questions
print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")


dataset meryam

In [ ]:
import pandas as pd
import json

# Load the TSV file
df = pd.read_csv('/content/myDB (3).tsv', sep='\t')
df.columns = df.columns.str.strip()

# Create a list to hold the JSON objects
data = []

# Process each row in the DataFrame
for _, row in df.iterrows():
    entry = {
        'question': row['question'],
        'options': {
            'A': row['Option_A'],
            'B': row['Option_B'],
            'C': row['Option_C'],
            'D': row['Option_D']
        },
        'correct_answer': row['Correct_Answer']
    }
    data.append(entry)

# Save to JSONL file
with open('/content/dataJson.jsonl', 'w') as f:
    for item in data:
        f.write(json.dumps(item) + '\n')


In [ ]:
import json

input_file = "/content/dataJson.jsonl"
output_file = "/content/cleaned_dataJson.jsonl"

# Function to sanitize options (convert all option values to strings)
def sanitize_options(options):
    for key, value in options.items():
        options[key] = str(value)  # Ensure all options are strings
    return options

# Open the original dataset and the output file
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        try:
            # Parse each line as JSON
            example = json.loads(line)

            # Sanitize the 'options' field
            if 'options' in example:
                example['options'] = sanitize_options(example['options'])

            # Write the sanitized example back to the new file
            outfile.write(json.dumps(example) + "\n")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue  # Skip any lines with JSON errors

print("Dataset has been cleaned and saved to", output_file)


In [ ]:


# Load the dataset from a JSONL file
dataset_path = '/content/cleaned_dataJson.jsonl'  # Change this path
mcq_dataset = []

with jsonlines.open(dataset_path) as reader:
    for obj in reader:
        mcq_dataset.append(obj)

# Alpaca-style prompt template
alpaca_prompt = "Question: {}\nOptions:\n{}\nAnswer:"

# Function to prepare the prompt
def prepare_prompt(question, options):
    # Ensure options is a dictionary and format correctly
    if isinstance(options, dict):
        options_text = "\n".join([f"{key}: {value}" for key, value in options.items()])
    else:
        options_text = options  # Handle the case where options are already formatted as a string
    return alpaca_prompt.format(question, options_text)

# Evaluation function using generation
def evaluate_mcq_with_generation(question, options, correct_answer, tokenizer, model):
    # Prepare the prompt
    prompt = prepare_prompt(question, options)

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the output and get the predicted answer
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the predicted answer (adapting based on the model's output format)
    if "Answer:" in generated_text:
        predicted_answer = generated_text.split("Answer:")[-1].strip()  # Extracting the answer after "Answer:"
    else:
        predicted_answer = generated_text.strip()  # In case the model doesn't output "Answer:"

    # Compare predicted answer with the correct answer
    return predicted_answer.lower() == correct_answer.lower()  # Case-insensitive comparison

# Evaluate the entire dataset
correct_predictions = 0
total_questions = len(mcq_dataset)

for mcq in mcq_dataset:
    question = mcq.get("question", "")
    options = mcq.get("options", "")
    correct_answer = mcq.get("correct_answer", "")

    # Check if correct_answer is valid before evaluating
    if isinstance(correct_answer, str):
        if evaluate_mcq_with_generation(question, options, correct_answer, tokenizer, model):
            correct_predictions += 1

# Calculate and print accuracy
accuracy = correct_predictions / total_questions
print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")


dataset demodel

In [ ]:
import pandas as pd
import json

# Load the TSV file
df = pd.read_csv('/content/cti-mcq.tsv',sep='\t')
df.columns = df.columns.str.strip()

# Create a list to hold the JSON objects
data = []

# Process each row in the DataFrame
for _, row in df.iterrows():
    entry = {
        'question': row['Question'],
        'options': {
            'A': row['Option A'],
            'B': row['Option B'],
            'C': row['Option C'],
            'D': row['Option D']
        },
        'correct_answer': row['GT']
    }
    data.append(entry)

# Save to JSONL file
with open('/content/dataJson.jsonl', 'w') as f:
    for item in data:
        f.write(json.dumps(item) + '\n')

In [ ]:
import json

input_file = "/content/dataJson.jsonl"
output_file = "/content/cleaned_dataJson.jsonl"

# Function to sanitize options (convert all option values to strings)
def sanitize_options(options):
    for key, value in options.items():
        options[key] = str(value)  # Ensure all options are strings
    return options

# Open the original dataset and the output file
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    for line in infile:
        try:
            # Parse each line as JSON
            example = json.loads(line)

            # Sanitize the 'options' field
            if 'options' in example:
                example['options'] = sanitize_options(example['options'])

            # Write the sanitized example back to the new file
            outfile.write(json.dumps(example) + "\n")
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue  # Skip any lines with JSON errors

print("Dataset has been cleaned and saved to", output_file)


In [ ]:


# Load the dataset from a JSONL file
dataset_path = '/content/cleaned_dataJson.jsonl'  # Change this path
mcq_dataset = []

with jsonlines.open(dataset_path) as reader:
    for obj in reader:
        mcq_dataset.append(obj)

# Alpaca-style prompt template
alpaca_prompt = "Question: {}\nOptions:\n{}\nAnswer:"

# Function to prepare the prompt
def prepare_prompt(question, options):
    # Ensure options is a dictionary and format correctly
    if isinstance(options, dict):
        options_text = "\n".join([f"{key}: {value}" for key, value in options.items()])
    else:
        options_text = options  # Handle the case where options are already formatted as a string
    return alpaca_prompt.format(question, options_text)

# Evaluation function using generation
def evaluate_mcq_with_generation(question, options, correct_answer, tokenizer, model):
    # Prepare the prompt
    prompt = prepare_prompt(question, options)

    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda" if torch.cuda.is_available() else "cpu")

    # Generate the output
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)

    # Decode the output and get the predicted answer
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the predicted answer (adapting based on the model's output format)
    if "Answer:" in generated_text:
        predicted_answer = generated_text.split("Answer:")[-1].strip()  # Extracting the answer after "Answer:"
    else:
        predicted_answer = generated_text.strip()  # In case the model doesn't output "Answer:"

    # Compare predicted answer with the correct answer
    return predicted_answer.lower() == correct_answer.lower()  # Case-insensitive comparison

# Evaluate the entire dataset
correct_predictions = 0
total_questions = len(mcq_dataset)

for mcq in mcq_dataset:
    question = mcq.get("question", "")
    options = mcq.get("options", "")
    correct_answer = mcq.get("correct_answer", "")

    # Check if correct_answer is valid before evaluating
    if isinstance(correct_answer, str):
        if evaluate_mcq_with_generation(question, options, correct_answer, tokenizer, model):
            correct_predictions += 1

# Calculate and print accuracy
accuracy = correct_predictions / total_questions
print(f"Evaluation Accuracy: {accuracy * 100:.2f}%")
